In [16]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
from time import sleep
from selenium import webdriver  
from selenium.webdriver.common.by import By
import re
import sys
import os
sys.path.append(os.path.abspath('../'))
from src import soporte_scrap as sc
import dotenv 
dotenv.load_dotenv()

True

In [17]:
driver = webdriver.Chrome()
url = 'https://www.pontemasfuerte.com'
driver.get(url)
driver.maximize_window()                                    #maximizo ventana
driver.find_element(By.CSS_SELECTOR, '#button1').click()        #boton cookies
sleep(5)
driver.find_element(By.XPATH, '//*[@id="vue-welcome-modal"]/div/div/header/p[2]').click()           #click ventana de cabecera
sleep(5)
driver.find_element(By.CSS_SELECTOR, '#pmfmenubar > div > div > ul > li:nth-child(1) > a').click()  #click todos los productos
#click para ver mas productos, podempos darle las veces que queramos...
driver.find_element(By.CSS_SELECTOR, '#wrapper > main > div.row.product-list > div > div.row > div.medium-8.large-10.columns > div:nth-child(6) > div > div > div').click()
sleep(5)
driver.find_element(By.CSS_SELECTOR, '#wrapper > main > div.row.product-list > div > div.row > div.medium-8.large-10.columns > div:nth-child(6) > div > div > div > span').click()
sleep(5)
driver.find_element(By.CSS_SELECTOR, '#wrapper > main > div.row.product-list > div > div.row > div.medium-8.large-10.columns > div:nth-child(6) > div > div > div > span').click()
sleep(5)
driver.find_element(By.CSS_SELECTOR, '#wrapper > main > div.row.product-list > div > div.row > div.medium-8.large-10.columns > div:nth-child(6) > div > div > div > span').click()

html_page = driver.page_source
sopa = BeautifulSoup(html_page)

In [3]:
Sopa_produc = sopa.findAll('div', class_ = 'item-lista not_mobile')


In [4]:
lista_productos = sc.scrap_fuerte(Sopa_produc)


In [5]:
lista_productos[1]

[' Whey Pure Fusion 2.3 Kg ',
 '76.30',
 '(127)',
 'Whey Pure Fusion de Amix es una proteína obtenida de aislado y concentrado de suero de una gran calidad con un alto contenido en aminoácidos ramificados y esenciales. Se trata de un proteína pura con una cantidad reducida de grasa y un bajo aporte de calorías. ',
 '4.7']

In [6]:
lista_productos
df = pd.DataFrame(lista_productos, columns=["Nombre y peso", "Precio", "cantidad de reseñas","decripcion",'rating'])

In [7]:
df[['Nombre', 'Peso']] = df['Nombre y peso'].str.extract(r'(.+)\s(\d+\s\w+)') 
#Eliminamos  la columna original 'Nombre y peso' 
df = df.drop(columns=['Nombre y peso'])

In [8]:
df.head()

,Precio,cantidad de reseñas,decripcion,rating,Nombre,Peso
0,69.90,(14),Real IsolatePRO presenta un elevado porcentaje...,4.4,Real IsolatePRO,2 Kg
1,76.30,(127),Whey Pure Fusion de Amix es una proteína obten...,4.7,NaN,NaN
2,29.90,(51),PurePRO Creatine es el suplemento perfecto par...,4.7,PurePRO Creatine (Creapure®),500 gr
3,91.80,(193),IsoPrime CFM Whey Isolate de Amix es una de la...,4.8,Isoprime CFM Isolate,2 Kg
4,16.92,(1),El suplemento L-Carnitine Pro de la marca PROF...,5.0,L-Carnitine Pro 3000 20 Serv. x,15 ml


In [9]:
df['cantidad de reseñas'] = df['cantidad de reseñas'].str.replace(r'[()]', '', regex=True)

In [10]:
df['Precio'] = df['Precio'].astype(float) 
df['cantidad de reseñas'] = df['cantidad de reseñas'].astype(float) 
df['rating'] = df['rating'].astype(float)

In [11]:
df.head(1)

,Precio,cantidad de reseñas,decripcion,rating,Nombre,Peso
0,69.9,14.0,Real IsolatePRO presenta un elevado porcentaje...,4.4,Real IsolatePRO,2 Kg


In [12]:
df[['Peso', 'Unidad']] = df['Peso'].str.extract(r'(\d+)\s*(\w+)')
df['Peso'] = df['Peso'].astype(float)

In [13]:
df['Peso'] = df['Peso'].fillna(0)

In [14]:
df

,Precio,cantidad de reseñas,decripcion,rating,Nombre,Peso,Unidad
0,69.90,14.0,Real IsolatePRO presenta un elevado porcentaje...,4.4,Real IsolatePRO,2.0,Kg
1,76.30,127.0,Whey Pure Fusion de Amix es una proteína obten...,4.7,NaN,0.0,NaN
2,29.90,51.0,PurePRO Creatine es el suplemento perfecto par...,4.7,PurePRO Creatine (Creapure®),500.0,gr
3,91.80,193.0,IsoPrime CFM Whey Isolate de Amix es una de la...,4.8,Isoprime CFM Isolate,2.0,Kg
4,16.92,1.0,El suplemento L-Carnitine Pro de la marca PROF...,5.0,L-Carnitine Pro 3000 20 Serv. x,15.0,ml
...,...,...,...,...,...,...,...
95,27.16,17.0,Iso Whey Zero de Biotech USA es un suplemento ...,4.8,Iso Whey Zero,500.0,gr
96,36.95,2.0,Total Recovery de Victory Endurance es un post...,4.5,Total Recovery,1250.0,gr
97,33.90,5.0,Micellar Casein de Big es una caseína micelar ...,5.0,Micellar Casein,1.0,Kg
98,34.51,35.0,100% Whey Protein Professional de Scitec Nutri...,4.6,100% Whey Protein Professional,920.0,gr


In [15]:
df.to_csv('../datos/datos_scrap_fuerte.csv', index=False)